<a href="https://colab.research.google.com/github/24p11/recode-scenario/blob/main/scenario_oncology_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Creation of oncology fictive clinical scenarios for medical documents generation

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Path to repository with references data
path_ref = "/content/drive/MyDrive/Referentials/"

In [4]:
# ICD10 cancer files
cim_cancer = pd.read_excel(path_ref + "REFERENTIEL_METHODE_DIM_CANCER_20140411.xls")
codes_cancer = cim_cancer.CIM10
codes_cancer_meta = ["C770","C771","C772","C773","C774","C775","C778","C779",
             "C780","C781","C782","C783","C784","C785","C786","C787","C788",
             "C790","C791","C792","C793","C794","C795","C796","C797","C798"]

racine_chimio = ['28Z07','17M05','17M06']
racine_radio = ["17K04","17K05","17K08","17K09","28Z10","28Z11","28Z18","28Z19",
                "28Z20","28Z21","28Z22","28Z23","28Z24","28Z25"]
racine_greffe = ["27Z02","27Z03","27Z04"]
racine_transfusion = ["28Z14"]
racine_soins_palliatifs = ["23Z02"]
racine_soins_stomies = ["06M17"]
racine_aphereses = ["28Z16"]
racine_deces = ["04M24"]
racine_bilan = ["23M03"]


In [5]:
df_situations = pd.read_excel(path_ref + "situations_cliniques.xlsx")
df_das = pd.read_excel(path_ref + "situations_cliniques_das.xlsx")
df_actes = pd.read_excel(path_ref + "situations_cliniques_actes.xlsx")
df_icd_official = pd.read_excel(path_ref + "cim_2024.xlsx")
df_icd_syn = pd.read_csv(path_ref + "cim_synonymes.csv").dropna()
df_chronique = pd.read_excel(path_ref + "Affections chroniques.xlsx",header=None,names=["code","chronic","libelle"])
df_stats_racines =  pd.read_excel(path_ref + "stat_racines.xlsx")
df_names = pd.read_csv(path_ref + "prenoms_nom_sexe.csv",sep=";").dropna()

New table DAS for chronic diseases with regrouping key = new_cage, racine, sexe, mode_sortie

In [6]:
df_das_racine = pd.read_csv(path_ref + "situations_das_racine_rgp_20250805.csv")
df_das_racine.rename(columns={"cage":"new_cage"},inplace=True)

In [7]:
df_stats_racines = df_stats_racines.assign(dsd = np.where(df_stats_racines.dsd.isna(),1,df_stats_racines.dsd))

In [8]:
#Add standard deviation of length of stay
df_situations = df_situations.merge(df_stats_racines[["racine","dsd"]],how="left")

Chronic disease are ranked 1,2 or 3 in the file ICD chronic.

In [9]:
icd_chronic = df_chronique[df_chronique.chronic.isin([1,2,3])].code.to_list()

Make a higher hierarchie for DAS :
- key between df_situations and df_das will be :
  * sexe,
  * new_cage,
  * categ_cim

In [10]:

df_situations = df_situations.assign(new_cage= np.where(df_situations.cage.isin(["[18-30[","[30-40[","[40-50["]),"[18-50[",
                                 np.where(df_situations.cage.isin(["[50-60[","[60-70[","[[70-80[","[80-["]),"[50-[","[0-18[")),
              categ_cim = df_situations.diag.str.slice(0,3))

df_das = df_das.assign(new_cage= np.where(df_das.cage.isin(["[18-30[","[30-40[","[40-50["]),"[18-50[",
                                 np.where(df_das.cage.isin(["[50-60[","[60-70[","[[70-80[","[80-["]),"[50-[","[0-18[")),
              categ_cim = df_das.diag.str.slice(0,3))



calculate new values for the number of time each DAS as been used for each key (sexe,new_cage,categ_cim)

In [11]:
df_das_new= df_das.groupby(["sexe","new_cage","das","categ_cim"]).agg({"nb_das":"sum"}).reset_index()

For cancer patients we will make 3 DAS dataframe :
- Metastasis : for each cancer (categ_cim) we will calculate in national database the number of case with a least one metastase and the mean number of metastase
- Chronicle desease : chronic desease from which we remove cancer
- Acute diseases which we will call complications

In [12]:
df_das_new_meta = df_das_new[df_das_new.das.isin(codes_cancer_meta)].reset_index(drop=True)
df_das_new_chronic = df_das_new[(~df_das_new.das.isin(codes_cancer_meta)) & (df_das_new.das.isin(icd_chronic)) & ~(df_das_new.das.str.slice(0,1)=="Z")& ~(df_das_new.das.isin(codes_cancer))  ].reset_index(drop=True)
#  df_das_new_complication = df_das_new[(~df_das_new.das.isin(codes_cancer_meta)) & ~(df_das_new.das.isin(icd_chronic)) & ~(df_das_new.das.str.slice(0,1)=="Z")].reset_index(drop=True)
df_das_new_complication = df_das_racine[(~df_das_racine.das.isin(codes_cancer_meta)) & ~(df_das_racine.das.isin(icd_chronic)) & ~(df_das_racine.das.str.slice(0,1)=="Z")].reset_index(drop=True)

In [13]:
df_das_new_meta_n_distinct = df_das_new_meta.groupby(["sexe","new_cage","categ_cim"]).agg(nb_distinct_das = ("das","nunique")).reset_index()
df_das_new_chronic_n_distinct = df_das_new_chronic.groupby(["sexe","new_cage","categ_cim"]).agg(nb_distinct_das = ("das","nunique")).reset_index()
df_das_new_complication_n_distinct = df_das_new_complication.groupby(["sexe","new_cage","racine","mode_sortie"]).agg(nb_distinct_das = ("das","nunique")).reset_index()

# Usefull functions

### Attribute DAS to each situations

Function to attribute DAS for each key (sexe,new_cage,categ_cim):
- Metastase :
  * choose randomly if patient has a metastase regarding metastasis distribution
  * choose randomly the number of metastasis in df_das_new_meta_n_distinct
  * choose randomly the metastic codes in df_das_new_meta regarding distribution of case (nb_das)
- Chronic disease :
  * choose randomly the number of chronic disease in regard with df_das_new_chronic_n_distinct taking into account a zero option
  * choose randomly the list of chronic disease in the df_das_new_chronic dataset regarding distribution of case (nb_das)
- Complication :
  * choose randomly the number of complications in regard with df_das_new_complication_n_distinct taking into account a zero option
  * choose randomly the list of complications in the df_das_new_complication dataset regarding distribution of case (nb_das)

For each ICD code at the end choose the right formulation in the

In [14]:
def sample_from_df(df_nb,df_values,max_nb,situation):

  df_values["libelle"] = ""

  nb_theo = df_nb.nb_distinct_das[(df_nb.new_cage == situation.new_cage) &
                           (df_nb.sexe == situation.sexe) &
                           (df_nb.categ_cim == situation.categ_cim) ]

  icd_theo = df_values[(df_values.new_cage == situation.new_cage) &
                  (df_values.sexe == situation.sexe) &
                  (df_values.categ_cim == situation.categ_cim) ]

  if len(nb_theo)==0:
    return pd.DataFrame(columns=df_values.columns)

  nb = np.minimum(np.random.randint(nb_theo, size=1)[0],max_nb)

  if nb > 0:
   icds = icd_theo.sample(nb, replace=False)
   icds["libelle"] = icds.das.apply(get_synonym)
   return icds
  else:
    return pd.DataFrame(columns=df_values.columns)


def sample_from_df(df_nb,df_values,col_weights,max_nb,distinct_chapter,situation):

  df_values["libelle"] = ""

  query = ' & '.join(['{}=={}'.format(k, "'{}'".format(v) if isinstance(v, str) else v) for k, v in situation.items() if k in df_nb.columns  ])

  nb_theo = df_nb.query(query).nb_distinct_das

  icd_theo = df_values.query(query)

  if len(nb_theo)==0:
    return pd.DataFrame(columns=df_values.columns)

  nb_max_das = np.minimum(nb_theo,max_nb)
  nb = np.random.randint(nb_max_das, size=1)[0]

  if nb > 0:

      #Codes are from different chapter of ICD10
      if distinct_chapter == True:

        icds=None
        chapter = []

        for i in range(1,nb+1):
            icds=pd.concat([icds,icd_theo[~(icd_theo.das.str.slice(0,1).isin(chapter))].sample(1, replace=False, weights =col_weights  )])
            chapter = icds.das.str.slice(0,1).to_list()

      else:
        icds = icd_theo.sample(nb, replace=False, weights =col_weights)

      icds["libelle"] = icds.das.apply(get_synonym)
      return icds


  else:
    return pd.DataFrame(columns=df_values.columns)

### Get synomym from ICD10 code

In [15]:
def get_synonym(code):

  if code in codes_cancer_meta:
    synonyms = df_icd_syn.dictionary_keys[(df_icd_syn.code==code) & ( df_icd_syn.dictionary_keys.str.contains("metastase"))]
  else:
    synonyms = df_icd_syn.dictionary_keys[df_icd_syn.code==code]

  #synonyms = df_icd_syn.dictionary_keys[df_icd_syn.code==code]
  if len(synonyms) > 0:
    return str(synonyms.sample(1).iloc[0])
  else:
    # Check if the code exists in the official ICD list
    official_libelle = df_icd_official.libelle[df_icd_official.code==code]
    if len(official_libelle) > 0:
      return str(official_libelle.iloc[0])
    else:
      return "" # Return empty string if code not found in official list

### Generate random date

In [28]:
import datetime
import random

def random_date(year,exclude_weekends=False):
  """Generates a random date in the year 2024.

  Args:
    exclude_weekends: If True, excludes Saturdays and Sundays.
  """
  while True:
    year = 2024
    month = random.randint(1, 12)
    # Get the number of days in the randomly chosen month
    if month == 2:
      if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0): # Check for leap year
        day = random.randint(1, 29)
      else:
        day = random.randint(1, 28)
    elif month in [4, 6, 9, 11]:
      day = random.randint(1, 30)
    else:
      day = random.randint(1, 31)

    random_date = datetime.date(year, month, day)

    if exclude_weekends:
      if random_date.weekday() < 5:  # Monday is 0, Sunday is 6
        break
    else:
      break
  return random_date

Generate a date between 2 dates for date of birth

In [29]:
def random_date_between(start_date, end_date):
  """Generates a random date between two datetime.date objects (inclusive)."""
  time_between_dates = end_date - start_date
  days_between_dates = time_between_dates.days
  random_number_of_days = random.randrange(days_between_dates + 1)
  random_date = start_date + datetime.timedelta(days=random_number_of_days)
  return random_date

### Generate date of entry and date of discharge from Mean Length of Stay (MLOS) and Standard Deviaton length of stay (DSLOS)

In [30]:
def get_dates_of_stay(type_hospitalisation, entry_mode,mols,sdlos):

  if type_hospitalisation == "HP" :
    date_entry = date_discharge = random_date(2024,exclude_weekends=False)

  else:
    los = int(np.round(np.random.normal(mols, sdlos, 1))[0])


    if entry_mode=="URGENCES":
     date_entry = random_date(2024,exclude_weekends=False)
    else :
      date_entry = random_date(2024,exclude_weekends=True)

    date_discharge = date_entry + datetime.timedelta(days=los)

  return date_entry, date_discharge



### Extraction first and second age from age categorie

In [31]:
import re

def extract_integers_from_cage(cage_string):
  """Extracts integer values from a string in the format "[x-y[" or "[x-[" and considers y as 90 if missing."""
  match = re.match(r"\[(\d+)-(\d+)\[", cage_string.strip())
  if match:
    return [int(match.group(1)), int(match.group(2))]
  else:
    # Handle the case for "[x-[" format like "[80-["
    match_single = re.match(r"\[(\d+)-\[", cage_string.strip())
    if match_single:
      return [int(match_single.group(1)), 90] # Consider missing y as 100
    else:
      return [] # Return empty list if no match



### Get random age from age categorie

In [32]:
def get_age(cage):
  age_min,age_max =extract_integers_from_cage(cage)
  return random.randint(age_min, age_max)



### Get first and last name

In [33]:
def get_names(gender):
  first_name = df_names.prenom[(df_names.sexe==gender) & (df_names.prenom.str.len()>3)].sample(1).iloc[0]
  last_name = df_names.nom[ (df_names.nom.str.len()>3)].sample(1).iloc[0]



  return first_name[0].upper() + first_name[1:].lower() ,last_name[0].upper() + last_name[1:].lower()

### Define "mode de prise en charge"

In [34]:
def define_md_pec(case):
    situa = ""
    code = 0

    if case["racine"] in racine_chimio and case["type_hosp"]  == "HP" :
      situa = "Prise en charge en hospitalisation de jour pour cure de chimiothérapie"
      code = 1

    elif case["racine"] in racine_chimio and case["type_hosp"]  == "HC":
      situa = "Prise en charge en hospitalisation complète pour cure de chimiothérapie"
      code = 2

    elif case["racine"] in racine_radio and case["type_hosp"]  == "HP":
      situa = "Prise en charge en hospitalisation de jour pour séance de radiothérapie"
      code = 3

    elif case["racine"] in racine_radio and case["type_hosp"]  == "HC":
      situa = "Prise en charge en hospitalisation complète pour réalisation du traitment de radiothérapie"
      code = 4

    elif case["racine"] in racine_greffe:
      situa = "Prise en charge pour " + case["libelle_racine"].lower()
      code = 5

    elif case["racine"] in racine_transfusion :
      situa = "Prise en charge pour " + case["libelle_racine"].lower()
      code = 6

    elif case["racine"] in racine_aphereses :
      situa = "Prise en charge pour " + case["libelle_racine"].lower()
      code = 7

    elif case["racine"] in racine_soins_palliatifs :
      actes = df_actes[(df_actes["racine"] == case["racine"]) &\
                       (df_actes["type_hosp"] == case["type_hosp"]) &\
                       (df_actes["sexe"] == case["sexe"]) &\
                       (df_actes["diag"] == case["diag"])]

      if len(actes) > 0:
        situa = "Prise en charge pour soins palliatifs avec " + actes.sample(1).iloc[0].copy()["libelle_long"].lower()
      else:
        situa = "Prise en charge pour soins palliatifs"

      code = 8

    elif case["racine"] in racine_soins_stomies:
      situa = "Prise en charge pour " + case["libelle_racine"].lower()
      code = 9

    elif (case["racine"] in racine_deces) | (case["mode_sortie"] == "DECES"):
      situa =  "Hospitalisation au cours de laquelle le patient est décédé"
      code = 10

    elif case["racine"][2:3] in ["C","K"] and case["type_hosp"]  == "HP" :
      situa =  "Prise en charge en chirugie ambulatoire pour "
      actes = df_actes[(df_actes["racine"] == case["racine"]) &\
                       (df_actes["type_hosp"] == case["type_hosp"]) &\
                       (df_actes["sexe"] == case["sexe"]) &\
                       (df_actes["diag"] == case["diag"])]

      if len(actes) > 0:
        situa += actes.sample(1).iloc[0].copy()["libelle_long"].lower()

      code = 11

    elif case["racine"][2:3] in ["C","K"] and case["type_hosp"]  == "HC" :
      situa =  "Prise en charge chirugicale en hospitalisation complète pour "
      actes = df_actes[(df_actes["racine"] == case["racine"]) &\
                       (df_actes["type_hosp"] == case["type_hosp"]) &\
                       (df_actes["sexe"] == case["sexe"]) &\
                       (df_actes["diag"] == case["diag"])]

      if len(actes) > 0:
        situa += actes.sample(1).iloc[0].copy()["libelle_long"].lower()

      code = 12

    else :
      option = np.random.choice(3, p=[0.4, 0.3, 0.3])
      if option == 0:
        situa = "Première hospitalisation pour découverte de cancer" # 40%
        code = 13
      elif option == 1:
        situa = "Première diagnostique et thérapeutique dans le cadre d'une rechute du cancer après traitement" # 30%
        code = 14
      else:
        situa =  "Hospitalisation pour bilan et surveillance du cancer" # 30%
        code = 15

    return (situa, code)

# Make clinical scenario from situations


Define all the necessary variables

In [35]:
scenario = {}
situation = df_situations.iloc[1000,:]
situation

,1000
diag,C040
cage,[60-70[
sexe,1
mode_entree,DOMICILE
mode_sortie,DOMICILE
type_hosp,HC
racine,03M07
gp,X09
ga,G092
nb,42


In [54]:
situation = df_situations.iloc[1587,:]
scenario = {}

if situation.diag in codes_cancer :
  scenario["diagnosis"] = situation.diag
else :
  scenario["diagnosis"] = get_synonym(situation.diag)

scenario["age"] = get_age(situation.cage)
scenario["metastases"] =  sample_from_df(df_das_new_meta_n_distinct,df_das_new_meta,"nb_das",4,False,situation).libelle.to_list()
scenario["chronic_diseases"] =   sample_from_df(df_das_new_chronic_n_distinct,df_das_new_chronic,"nb_das",5,False,situation).libelle.to_list()
scenario["complications"] = sample_from_df(df_das_new_complication_n_distinct,df_das_new_complication,"nb_situations",5,True,situation).libelle.to_list()
scenario["date_entry"],scenario["date_discharge"] = get_dates_of_stay(situation.type_hosp,situation.mode_entree,situation.dms,situation.dsd)
scenario["date_of_birth"] = random_date_between( scenario["date_entry"] - datetime.timedelta(days = 365*(scenario["age"]+1)) , scenario["date_entry"] - datetime.timedelta(days = 365*(scenario["age"])))
scenario["md_pec"] , scenario["cd_md_pec"] = define_md_pec(situation)
scenario["first_name"] , scenario["laste_name"] = get_names(situation.sexe)
scenario["first_name_med"] , scenario["laste_name_med"] = get_names(random.randint(1, 2))

In [56]:
situation.diag

'C180'

In [ ]:
def make_scenario(situation):
  scenario = {}

  scenario["diagnosis"] = get_synonym(situation.diag)
  scenario["age"] = get_age(situation.cage)
  scenario["metastases"] =  sample_from_df(df_das_new_meta_n_distinct,df_das_new_meta,5,False,situation).libelle.to_list()
  scenario["chronic_diseases"] =   sample_from_df(df_das_new_chronic_n_distinct,df_das_new_chronic,5,False,situation).libelle.to_list()
  scenario["complications"] = sample_from_df(df_das_new_complication_n_distinct,df_das_new_complication,5,True,situation).libelle.to_list()
  scenario["date_entry"],scenario["date_discharge"] = get_dates_of_stay(situation.type_hosp,situation.mode_entree,situation.dms,situation.dsd)
  scenario["date_of_birth"] = random_date_between( scenario["date_entry"] - datetime.timedelta(days = 365*(scenario["age"]+1)) , scenario["date_entry"] - datetime.timedelta(days = 365*(scenario["age"])))
  scenario["md_pec"] , scenario["cd_md_pec"] = define_md_pec(situation)
  scenario["first_name"] , scenario["laste_name"] = get_names(situation.sexe)
  scenario["first_name_med"] , scenario["laste_name_med"] = get_names(random.randint(1, 2))

  return(scenario)


In [ ]:
n_sample = 20

df_situations_scenario = df_situations.sample(n_sample, replace=False,weights="nb").reset_index(drop=True)



In [ ]:
df_situations_scenario["scenario"] = df_situations_scenario.apply(make_scenario,axis=1)

In [ ]:
df_situations_scenario["scenario"][6]

{'diagnosis': 'gastrite atrophique',
 'age': 34,
 'metastases': [],
 'chronic_diseases': ['hypertension arterielle commune accompagnee'],
 'complications': ['degenerescence moelle combinee carence en vitamine b avec anemie pernicieuse',
  'douleur abdominale',
  'gastrite chronique erosive',
  'syndrome de dependance au tabac utilisation actuelle',
  'selles liquides abondantes'],
 'date_entry': datetime.date(2024, 8, 30),
 'date_discharge': datetime.date(2024, 8, 30),
 'date_of_birth': datetime.date(1990, 5, 31),
 'md_pec': 'Prise en charge en chirugie ambulatoire pour examen histopathologique de biopsies étagées de 3 structures anatomiques',
 'cd_md_pec': 11,
 'first_name': 'Simone',
 'laste_name': 'Rouxel',
 'first_name_med': 'Claude',
 'laste_name_med': 'Dewerdt'}